In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
import pprint
import spicy
from spicy import stats
import numpy as np
import math


In [24]:
df = pd.read_csv('International Astronaut Database.csv')

In [25]:
# Split Flights into separate rows and extract FlightCode and Year
import re

# parser function
def parse_flights_cell(s):
    parts = [p.strip() for p in str(s).split(',') if p.strip()]
    entries = []
    for p in parts:
        m = re.match(r"^(?P<code>.*?)\s*\((?P<year>\d{4})\)\s*$", p)
        if m:
            code = m.group('code').strip()
            year = int(m.group('year'))
        else:
            y = re.search(r"(\d{4})", p)
            year = int(y.group(1)) if y else None
            code = re.sub(r"\(.*\)", "", p).strip()
        entries.append({'FlightCode': code, 'Year': year, 'OriginalFlight': p})
    return entries

# apply to DataFrame 'df' loaded earlier in the notebook
if 'df' in globals():
    df['Flights_list'] = df['Flights'].fillna('').apply(parse_flights_cell)
    df_flights = df.explode('Flights_list').reset_index(drop=True)
    df_flights = df_flights[df_flights['Flights_list'].map(lambda x: x is not None)]
    df_flights[['FlightCode','Year','OriginalFlight']] = df_flights['Flights_list'].apply(lambda d: pd.Series([d['FlightCode'], d['Year'], d['OriginalFlight']]))
    # Optionally drop the helper column
    # df_flights = df_flights.drop(columns=['Flights_list'])
    print('Created exploded DataFrame with FlightCode and Year columns. Rows:', len(df_flights))
else:
    print("DataFrame 'df' not found in the notebook. Make sure you've run the cell that loads the CSV.")

# Optional: save to CSV
# df_flights.to_csv('astronaut_flights_exploded.csv', index=False)


Created exploded DataFrame with FlightCode and Year columns. Rows: 1294


In [26]:
def convert_flight_time_to_minutes(flight_time_str):
    if pd.isna(flight_time_str):
        return 0
    try:
        d, h, m = map(int, flight_time_str.split(':'))
        total_minutes = d * 24 * 60 + h * 60 + m
        return total_minutes
    except Exception as e:
        print(f"Error parsing flight time '{flight_time_str}': {e}")
        return 0

In [27]:
df_flights['Total Flight Time (minute)'] = df_flights['Total Flight Time (ddd:hh:mm)'].apply(convert_flight_time_to_minutes)

In [28]:
df_flights.drop(["Flights", "Flights_list", "OriginalFlight", "Total Flight Time (ddd:hh:mm)"], axis=1, inplace=True)

In [31]:
df_flights['Prev_flights'] = df_flights.groupby('Name').cumcount()

In [32]:
df_flights

,Name,Country,Gender,Total Flights,FlightCode,Year,Total Flight Time (minute),Prev_flights
0,Abdul Ahad Mohmand,Afghanistan,Man,1,Soyuz TM-6,1988,12746,0
1,Akihiko Hoshide,Japan,Man,3,STS-124,2008,2522785,0
2,Akihiko Hoshide,Japan,Man,3,Soyuz TMA-05M,2012,2522785,1
3,Akihiko Hoshide,Japan,Man,3,SpaceX Crew-2,2021,2522785,2
4,Alan Bean,United States,Man,2,Apollo 12,1969,100305,0
...,...,...,...,...,...,...,...,...
1289,Yury Usachov,Russia,Man,4,Soyuz TM-23,1996,796224,1
1290,Yury Usachov,Russia,Man,4,STS-101,2000,796224,2
1291,Yury Usachov,Russia,Man,4,STS-102,2001,796224,3
1292,Zhai Zhigang,China,Man,1,Shenzhou 7,2008,4106,0
